In [1]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info = True, as_supervised = True)
print(info)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteV66EPR/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteV66EPR/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteV66EPR/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.
tfds.core.DatasetInfo(
    name='imdb_reviews',
    full_name='imdb_reviews/plain_text/1.0.0',
    description="""
    Large Movie Review Dataset. This is a dataset for binary sentiment
    classification containing substantially more data than previous benchmark
    datasets. We provide a set of 25,000 highly polar movie reviews for training,
    and 25,000 for testing. There is additional unlabeled data for use as well.
    """,
    config_description="""
    Plain text
    """,
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    data_path=PosixGPath('/tmp/tmpytps3zn1tfds'),
    file_format=tfrecord,
    download_size=80.23 MiB,
    dataset_size=129.83 MiB,
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
        'text': Text(shape=(), dtype=string),
    }),
    supervised_keys=('tex

In [6]:
imdb

{Split('train'): <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 Split('test'): <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 Split('unsupervised'): <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>}

In [7]:
for example in imdb["train"].take(3):
    print(example)

(<tf.Tensor: shape=(), dtype=string, numpy=b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.">, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on

In [19]:
import numpy as np
train_data, test_data = imdb["train"], imdb["test"]

training_sentences, training_labels = [], []
testing_sentences, testing_labels = [], []

In [20]:
for sentence, label in train_data:
    training_sentences.append(sentence.numpy().decode("utf8"))
    training_labels.append(label.numpy())
for sentence, label in test_data:
    testing_sentences.append(sentence.numpy().decode("utf8"))
    testing_labels.append(label.numpy())

In [21]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [22]:
vocab_size = 10000
max_length = 120
embedding_dim = 16
trunct_type = "post"
oov_tok = "<OOV>"

In [23]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

In [24]:
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen = max_length, truncating = trunct_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen = max_length, truncating = trunct_type)

In [33]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation = "relu"),
    tf.keras.layers.Dense(1, activation = "sigmoid")
])
model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 120, 16)           160000    
                                                                 
 flatten_5 (Flatten)         (None, 1920)              0         
                                                                 
 dense_10 (Dense)            (None, 6)                 11526     
                                                                 
 dense_11 (Dense)            (None, 1)                 7         
                                                                 
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [34]:
num_epochs = 20
model.fit(padded, training_labels_final, epochs = num_epochs, validation_data = (testing_padded, testing_labels_final))

Epoch 1/20
782/782 [==============================] - 5s 6ms/step - loss: 0.4942 - accuracy: 0.7450 - val_loss: 0.3887 - val_accuracy: 0.8236
Epoch 2/20
782/782 [==============================] - 4s 6ms/step - loss: 0.2469 - accuracy: 0.9035 - val_loss: 0.4073 - val_accuracy: 0.8216
Epoch 3/20
782/782 [==============================] - 5s 6ms/step - loss: 0.1081 - accuracy: 0.9700 - val_loss: 0.4925 - val_accuracy: 0.8129
Epoch 4/20
782/782 [==============================] - 4s 6ms/step - loss: 0.0312 - accuracy: 0.9955 - val_loss: 0.5828 - val_accuracy: 0.8102
Epoch 5/20
782/782 [==============================] - 4s 6ms/step - loss: 0.0086 - accuracy: 0.9993 - val_loss: 0.6972 - val_accuracy: 0.8057
Epoch 6/20
782/782 [==============================] - 4s 5ms/step - loss: 0.0027 - accuracy: 0.9999 - val_loss: 0.7580 - val_accuracy: 0.8067
Epoch 7/20
782/782 [==============================] - 4s 5ms/step - loss: 0.0014 - accuracy: 0.9999 - val_loss: 0.8528 - val_accuracy: 0.8017
Epoch 

In [37]:
model.evaluate(testing_padded, testing_labels_final)
#since the model used the validation set during training, the estimation will be very optimistic

782/782 [==============================] - 1s 2ms/step - loss: 1.3632 - accuracy: 0.8091


[1.3632011413574219, 0.8091199994087219]

In [43]:
embedding_layer = model.layers[0]
embedding_weights = embedding_layer.get_weights()[0]
embedding_weights.shape

(10000, 16)

In [44]:
reverse_word_index = tokenizer.index_word

In [45]:
import io
out_v = io.open("vecs.tsv", "w", encoding = "utf-8")
out_m = io.open("meta.tsv", "w", encoding = "utf-8")
for word_num in range(1, vocab_size):
    word_name = reverse_word_index[word_num]
    word_embedding = embedding_weights[word_num]
    out_m.write(word_name + "\n")
    out_v.write('\t'.join([str(x) for x in word_embedding]) + "\n")
out_v.close()
out_m.close()

In [46]:
# Import files utilities in Colab
try:
  from google.colab import files
except ImportError:
  pass

# Download the files
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')